In [1]:
import glob, re
import numpy as np
import pandas as pd
from sklearn import *
from datetime import datetime
from xgboost import XGBRegressor
from keras.layers import Embedding, Input, Dense
import keras
import keras.backend as K
import matplotlib.pyplot as plt
from keras.models import Model
from keras.layers import Input, Dense
from pandas import concat
import glob, re
from sklearn.ensemble import RandomForestRegressor
from numpy import median
import xgboost as xgb
import warnings

/Users/mayanksoni/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/mayanksoni/anaconda/lib/python2.7/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
/Users/mayanksoni/anaconda/lib/python2.7/site-packages/sklearn/learning_curve.py:22: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. This module will be rem

In [2]:
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 1000)

height has been deprecated.



In [3]:
path = "/Users/mayanksoni/Desktop/Kaggle /Restaurant Visitor Forecasting"

In [4]:
#All the files are loaded into one dictionary
data = {'tra': pd.read_csv(path+"/air_visit_data.csv"),
    'as': pd.read_csv(path+"/air_store_info.csv"),
    'hs': pd.read_csv(path+"/hpg_store_info.csv"),
    'ar': pd.read_csv(path+"/air_reserve.csv"),
    'hr': pd.read_csv(path+"/hpg_reserve.csv"),
    'id': pd.read_csv(path+"/store_id_relation.csv"),
    'tes': pd.read_csv(path+"/sample_submission.csv"),
    'hol': pd.read_csv(path+"/date_info.csv").rename(columns={'calendar_date':'visit_date'})}

In [5]:
data['hr'] = pd.merge(data['hr'], data['id'], how='inner', on=['hpg_store_id'])

In [6]:
for df in ['ar','hr']:
    data[df]['visit_datetime'] = pd.to_datetime(data[df]['visit_datetime'])
    data[df]['visit_datetime'] = data[df]['visit_datetime'].dt.date
    data[df]['reserve_datetime'] = pd.to_datetime(data[df]['reserve_datetime'])
    data[df]['reserve_datetime'] = data[df]['reserve_datetime'].dt.date
    data[df]['reserve_datetime_diff'] = data[df].apply(lambda r: (r['visit_datetime'] - r['reserve_datetime']).days, axis=1)
    tmp1 = data[df].groupby(['air_store_id','visit_datetime'], as_index=False)[['reserve_datetime_diff', 'reserve_visitors']].sum().rename(columns={'visit_datetime':'visit_date', 'reserve_datetime_diff': 'rs1', 'reserve_visitors':'rv1'})
    tmp2 = data[df].groupby(['air_store_id','visit_datetime'], as_index=False)[['reserve_datetime_diff', 'reserve_visitors']].mean().rename(columns={'visit_datetime':'visit_date', 'reserve_datetime_diff': 'rs2', 'reserve_visitors':'rv2'})
    data[df] = pd.merge(tmp1, tmp2, how='inner', on=['air_store_id','visit_date'])

In [7]:
data['tra']['visit_date'] = pd.to_datetime(data['tra']['visit_date'])
data['tra']['dow'] = data['tra']['visit_date'].dt.dayofweek
data['tra']['doy'] = data['tra']['visit_date'].dt.dayofyear
data['tra']['year'] = data['tra']['visit_date'].dt.year
data['tra']['month'] = data['tra']['visit_date'].dt.month
data['tra']['week'] = data['tra']['visit_date'].dt.week
data['tra']['visit_date'] = data['tra']['visit_date'].dt.date

In [8]:
data['tes']['visit_date'] = data['tes']['id'].map(lambda x: str(x).split('_')[2])
data['tes']['air_store_id'] = data['tes']['id'].map(lambda x: '_'.join(x.split('_')[:2]))
data['tes']['visit_date'] = pd.to_datetime(data['tes']['visit_date'])
data['tes']['dow'] = data['tes']['visit_date'].dt.dayofweek
data['tes']['doy'] = data['tes']['visit_date'].dt.dayofyear
data['tes']['year'] = data['tes']['visit_date'].dt.year
data['tes']['month'] = data['tes']['visit_date'].dt.month
data['tes']['week'] = data['tes']['visit_date'].dt.week
data['tes']['visit_date'] = data['tes']['visit_date'].dt.date

In [9]:
unique_stores = data['tes']['air_store_id'].unique()
stores = pd.concat([pd.DataFrame({'air_store_id': unique_stores, 'dow': [i]*len(unique_stores)}) for i in range(7)], axis=0, ignore_index=True).reset_index(drop=True)

In [10]:
#sure it can be compressed...
tmp = data['tra'].groupby(['air_store_id','dow'], as_index=False)['visitors'].min().rename(columns={'visitors':'min_visitors'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow']) 
tmp = data['tra'].groupby(['air_store_id','dow'], as_index=False)['visitors'].mean().rename(columns={'visitors':'mean_visitors'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow'])
tmp = data['tra'].groupby(['air_store_id','dow'], as_index=False)['visitors'].median().rename(columns={'visitors':'median_visitors'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow'])
#tmp = data['tra'].groupby(['air_store_id','dow'], as_index=False)['visitors'].max().rename(columns={'visitors':'max_visitors'})
#stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow'])
tmp = data['tra'].groupby(['air_store_id','dow'], as_index=False)['visitors'].count().rename(columns={'visitors':'count_observations'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow']) 

stores = pd.merge(stores, data['as'], how='left', on=['air_store_id']) 

In [11]:
# NEW FEATURES FROM Georgii Vyshnia
stores['air_genre_name'] = stores['air_genre_name'].map(lambda x: str(str(x).replace('/',' ')))
stores['air_area_name'] = stores['air_area_name'].map(lambda x: str(str(x).replace('-',' ')))
lbl = preprocessing.LabelEncoder()
for i in range(10):
    stores['air_genre_name'+str(i)] = lbl.fit_transform(stores['air_genre_name'].map(lambda x: str(str(x).split(' ')[i]) if len(str(x).split(' '))>i else ''))
    stores['air_area_name'+str(i)] = lbl.fit_transform(stores['air_area_name'].map(lambda x: str(str(x).split(' ')[i]) if len(str(x).split(' '))>i else ''))
stores['air_genre_name'] = lbl.fit_transform(stores['air_genre_name'])
stores['air_area_name'] = lbl.fit_transform(stores['air_area_name'])

In [12]:
data['hol']['visit_date'] = pd.to_datetime(data['hol']['visit_date'])
data['hol']['day_of_week'] = lbl.fit_transform(data['hol']['day_of_week'])
data['hol']['visit_date'] = data['hol']['visit_date'].dt.date
train = pd.merge(data['tra'], data['hol'], how='left', on=['visit_date']) 
test = pd.merge(data['tes'], data['hol'], how='left', on=['visit_date']) 

In [13]:
train = pd.merge(train, stores, how='inner', on=['air_store_id','dow']) 
test = pd.merge(test, stores, how='left', on=['air_store_id','dow'])

In [14]:
# data['ar'].rename(columns={'visit_datetime':'visit_date'},inplace = True)

In [15]:
for df in ['ar','hr']:
    train = pd.merge(train, data[df], how='left', on=['air_store_id','visit_date']) 
    test = pd.merge(test, data[df], how='left', on=['air_store_id','visit_date'])

In [16]:
train['id'] = train.apply(lambda r: '_'.join([str(r['air_store_id']), str(r['visit_date'])]), axis=1)

In [17]:
train['total_reserv_sum'] = train['rv1_x'] + train['rv1_y']
train['total_reserv_mean'] = (train['rv2_x'] + train['rv2_y']) / 2
train['total_reserv_dt_diff_mean'] = (train['rs2_x'] + train['rs2_y']) / 2

In [18]:
test['total_reserv_sum'] = test['rv1_x'] + test['rv1_y']
test['total_reserv_mean'] = (test['rv2_x'] + test['rv2_y']) / 2
test['total_reserv_dt_diff_mean'] = (test['rs2_x'] + test['rs2_y']) / 2

In [19]:
# NEW FEATURES FROM JMBULL
train['date_int'] = train['visit_date'].apply(lambda x: x.strftime('%Y%m%d')).astype(int)
test['date_int'] = test['visit_date'].apply(lambda x: x.strftime('%Y%m%d')).astype(int)
train['var_max_lat'] = train['latitude'].max() - train['latitude']
train['var_max_long'] = train['longitude'].max() - train['longitude']
test['var_max_lat'] = test['latitude'].max() - test['latitude']
test['var_max_long'] = test['longitude'].max() - test['longitude']

In [20]:
# NEW FEATURES FROM Georgii Vyshnia
train['lon_plus_lat'] = train['longitude'] + train['latitude'] 
test['lon_plus_lat'] = test['longitude'] + test['latitude']

In [21]:
lbl = preprocessing.LabelEncoder()
train['air_store_id2'] = lbl.fit_transform(train['air_store_id'])
test['air_store_id2'] = lbl.transform(test['air_store_id'])

In [22]:
#Extracting day of the month from date
train['date'] = pd.to_datetime(train['visit_date'])
train['date'] = train['date'].dt.day
test['date'] = pd.to_datetime(test['visit_date'])
test['date'] = test['date'].dt.day

In [23]:
# train = train.loc[(train['date_int'] >= 20170101)&(train['date_int'] <=20170423 )]

In [24]:
#daily average of store
temp = train.groupby(['air_store_id','date'],as_index=False)['visitors'].mean().rename(columns={'visitors':'dly_avg'})
train = pd.merge(train,temp,on = ('air_store_id','date'))
test = pd.merge(test,temp,on = ('air_store_id','date'),how='left')
#daily median of store
temp = train.groupby(['air_store_id','date'],as_index=False)['visitors'].mean().rename(columns={'visitors':'dly_median'})
train = pd.merge(train,temp,on = ('air_store_id','date'))
test = pd.merge(test,temp,on = ('air_store_id','date'),how='left')
# #daily min of store
temp = train.groupby(['air_store_id','date'],as_index=False)['visitors'].min().rename(columns={'visitors':'dly_min'})
train = pd.merge(train,temp,on = ('air_store_id','date'))
test = pd.merge(test,temp,on = ('air_store_id','date'),how='left')
#daily max of store
temp = train.groupby(['air_store_id','date'],as_index=False)['visitors'].max().rename(columns={'visitors':'dly_max'})
train = pd.merge(train,temp,on = ('air_store_id','date'))
test = pd.merge(test,temp,on = ('air_store_id','date'),how='left')

In [25]:
#other longitude lattitude feature
#Lets add some new lat long features
train['lat-lon'] = train['latitude'] - train['longitude']
train['lat/lon'] = train['latitude'] / train['longitude']
train['latintolon'] = train['latitude']*train['longitude']
test['lat-lon'] = test['latitude'] - test['longitude']
test['lat/lon'] = test['latitude'] / test['longitude']
test['latintolon'] = test['latitude']*test['longitude']

In [26]:
#monthly average
temp = train.groupby(['air_store_id','month'],as_index=False)['visitors'].mean().rename(columns={'visitors':'mnth_avg'})
train = pd.merge(train,temp,on = ('air_store_id','month'))
test = pd.merge(test,temp,on = ('air_store_id','month'),how='left')
#monthly median
temp = train.groupby(['air_store_id','month'],as_index=False)['visitors'].median().rename(columns={'visitors':'mnth_median'})
train = pd.merge(train,temp,on = ('air_store_id','month'))
test = pd.merge(test,temp,on = ('air_store_id','month'),how='left')
#monthly min
temp = train.groupby(['air_store_id','month'],as_index=False)['visitors'].min().rename(columns={'visitors':'mnth_min'})
train = pd.merge(train,temp,on = ('air_store_id','month'))
test = pd.merge(test,temp,on = ('air_store_id','month'),how='left')
#monthly max
temp = train.groupby(['air_store_id','month'],as_index=False)['visitors'].max().rename(columns={'visitors':'mnth_max'})
train = pd.merge(train,temp,on = ('air_store_id','month'))
test = pd.merge(test,temp,on = ('air_store_id','month'),how='left')

In [27]:
#monthly_avg - Daily_avg
train['mnthly-avg_dly'] = train['dly_avg'] - train['mnth_avg']
test['mnthly-avg_dly'] = test['dly_avg'] - test['mnth_avg']

In [28]:
#average visitors to a restaurant - Monthly , weekly , quaterly and  the substraction
temp = train.groupby(['air_store_id'],as_index=False)['visitors'].mean().rename(columns={'visitors':'restaurant_avg'})
train = pd.merge(train,temp,on = ('air_store_id'))
test = pd.merge(test,temp,on = ('air_store_id'),how = 'left')
#median visitors to a restaurant - Monthly , weekly , quaterly and  the substraction
temp = train.groupby(['air_store_id'],as_index=False)['visitors'].median().rename(columns={'visitors':'restaurant_median'})
train = pd.merge(train,temp,on = ('air_store_id'))
test = pd.merge(test,temp,on = ('air_store_id'),how = 'left')
#min visitors to a restaurant - Monthly , weekly , quaterly and  the substraction
temp = train.groupby(['air_store_id'],as_index=False)['visitors'].min().rename(columns={'visitors':'restaurant_min'})
train = pd.merge(train,temp,on = ('air_store_id'))
test = pd.merge(test,temp,on = ('air_store_id'),how = 'left')
#max visitors to a restaurant - Monthly , weekly , quaterly and  the substraction
temp = train.groupby(['air_store_id'],as_index=False)['visitors'].max().rename(columns={'visitors':'restaurant_max'})
train = pd.merge(train,temp,on = ('air_store_id'))
test = pd.merge(test,temp,on = ('air_store_id'),how = 'left')

In [29]:
#overall average - daily average
train['mnthly-avg_dly'] = train['restaurant_avg'] - train['dly_avg']
test['mnthly-avg_dly'] = test['restaurant_avg'] - test['dly_avg']

In [30]:
#overall average - monthly average
train['avg_dly_mnth'] = train['restaurant_avg'] - train['mnth_avg']
test['avg_dly_mnth'] = test['restaurant_avg'] - test['mnth_avg']

In [31]:
#average visit for a area per day.
train_temp = train.groupby(['month','date','air_area_name'],as_index=False)['visitors'].mean().rename(columns={'visitors':'mnth_dt_avg_by_area'})
train = pd.merge(train,train_temp,on = ('month','date','air_area_name'))
test = pd.merge(test,train_temp,on = ('month','date','air_area_name'),how='left')
#median visit for a area per day.
train_temp = train.groupby(['month','date','air_area_name'],as_index=False)['visitors'].median().rename(columns={'visitors':'mnth_dt_median_by_area'})
train = pd.merge(train,train_temp,on = ('month','date','air_area_name'))
test = pd.merge(test,train_temp,on = ('month','date','air_area_name'),how='left')
#min visit for a area per day.
train_temp = train.groupby(['month','date','air_area_name'],as_index=False)['visitors'].min().rename(columns={'visitors':'mnth_dt_min_by_area'})
train = pd.merge(train,train_temp,on = ('month','date','air_area_name'))
test = pd.merge(test,train_temp,on = ('month','date','air_area_name'),how='left')
#min visit for a area per day.
train_temp = train.groupby(['month','date','air_area_name'],as_index=False)['visitors'].max().rename(columns={'visitors':'mnth_dt_max_by_area'})
train = pd.merge(train,train_temp,on = ('month','date','air_area_name'))
test = pd.merge(test,train_temp,on = ('month','date','air_area_name'),how='left')

In [32]:
#average visitors per day of week- This has been calculated already.
temp = train.groupby(['air_store_id','dow'],as_index=False)['visitors'].mean().rename(columns={'visitors':'dow_avg'})
train = pd.merge(train,temp,on = ('air_store_id','dow'))
test = pd.merge(test,temp,on = ('air_store_id','dow'),how = 'left')

In [33]:
#average visitors per day of week by area
train_temp = train.groupby(['month','dow','air_area_name'],as_index=False)['visitors'].mean().rename(columns={'visitors':'avg_vstr_wkday_by_area'})
train = pd.merge(train,train_temp,on = ('month','dow','air_area_name'))
test = pd.merge(test,train_temp,on = ('month','dow','air_area_name'),how='left')
#median visitors per day of week by area
train_temp = train.groupby(['month','dow','air_area_name'],as_index=False)['visitors'].median().rename(columns={'visitors':'median_vstr_wkday_by_area'})
train = pd.merge(train,train_temp,on = ('month','dow','air_area_name'))
test = pd.merge(test,train_temp,on = ('month','dow','air_area_name'),how='left')
#min visitors per day of week by area
train_temp = train.groupby(['month','dow','air_area_name'],as_index=False)['visitors'].min().rename(columns={'visitors':'min_vstr_wkday_by_area'})
train = pd.merge(train,train_temp,on = ('month','dow','air_area_name'))
test = pd.merge(test,train_temp,on = ('month','dow','air_area_name'),how='left')
#max visitors per day of week by area
train_temp = train.groupby(['month','dow','air_area_name'],as_index=False)['visitors'].max().rename(columns={'visitors':'max_vstr_wkday_by_area'})
train = pd.merge(train,train_temp,on = ('month','dow','air_area_name'))
test = pd.merge(test,train_temp,on = ('month','dow','air_area_name'),how='left')

In [34]:
#average weekly visitors per cuisine
train_temp = train.groupby(['air_genre_name','dow'],as_index=False)['visitors'].mean().rename(columns={'visitors':'avg_vstr_wkday_by_cuisine'})
train = pd.merge(train,train_temp,on = ('air_genre_name','dow'))
test = pd.merge(test,train_temp,on = ('air_genre_name','dow'),how='left')
#median weekly visitors per cuisine
train_temp = train.groupby(['air_genre_name','dow'],as_index=False)['visitors'].median().rename(columns={'visitors':'median_vstr_wkday_by_cuisine'})
train = pd.merge(train,train_temp,on = ('air_genre_name','dow'))
test = pd.merge(test,train_temp,on = ('air_genre_name','dow'),how='left')
#minimum weekly visitors per cuisine
train_temp = train.groupby(['air_genre_name','dow'],as_index=False)['visitors'].min().rename(columns={'visitors':'min_vstr_wkday_by_cuisine'})
train = pd.merge(train,train_temp,on = ('air_genre_name','dow'))
test = pd.merge(test,train_temp,on = ('air_genre_name','dow'),how='left')
#maximum weekly visitors per cuisine
train_temp = train.groupby(['air_genre_name','dow'],as_index=False)['visitors'].max().rename(columns={'visitors':'max_vstr_wkday_by_cuisine'})
train = pd.merge(train,train_temp,on = ('air_genre_name','dow'))
test = pd.merge(test,train_temp,on = ('air_genre_name','dow'),how='left')

In [35]:
#Average of averages
train['avgofavg'] = train[['dly_avg','mean_visitors','mnth_avg','restaurant_avg']].mean(axis=1)
test['avgofavg'] = test[['dly_avg','mean_visitors','mnth_avg','restaurant_avg']].mean(axis=1)
#median of averages
train['medianofavg'] = train[['dly_avg','mean_visitors','mnth_avg','restaurant_avg']].median(axis=1)
test['medianofavg'] = test[['dly_avg','mean_visitors','mnth_avg','restaurant_avg']].median(axis=1)

In [36]:
#Import after adding lag features in train and test
train_temp = pd.read_excel("/Users/mayanksoni/Desktop/Kaggle /Restaurant Visitor Forecasting/train/sheet2.xlsx")
train = pd.merge(train,train_temp,on = ('air_store_id','date'))
test = pd.merge(test,train_temp,on = ('air_store_id','date'),how='left')

In [37]:
#number of restaurants by area
train_temp = train.groupby("air_area_name",as_index=False).agg({"air_store_id": pd.Series.nunique}).rename(columns={'air_store_id':'restaurants_by_area'})
train = pd.merge(train,train_temp,on = ('air_area_name'))
test = pd.merge(test,train_temp,on = ('air_area_name'),how='left')

In [38]:
#rating feature
train_temp = pd.read_csv("/Users/mayanksoni/Desktop/Kaggle /Restaurant Visitor Forecasting/train/rating.csv")
train = pd.merge(train,train_temp,on = ('air_store_id'))
test = pd.merge(test,train_temp,on = ('air_store_id'),how='left')

In [39]:
#Loading back with lag features.
#Let's sort it first
train = train.sort_values(['air_store_id','visit_date'])
test = test.sort_values(['air_store_id','visit_date'])
#Creating lag features
#lag1 - lag7
train['lag_1'] = train['visitors'].shift(48)
train['lag_2'] = train['visitors'].shift(47)
train['lag_3'] = train['visitors'].shift(46)
train['lag_4'] = train['visitors'].shift(45)
train['lag_5'] = train['visitors'].shift(44)
train['lag_6'] = train['visitors'].shift(43)
test['lag_1'] = test['visitors'].shift(48)
test['lag_2'] = test['visitors'].shift(47)
test['lag_3'] = test['visitors'].shift(46)
test['lag_4'] = test['visitors'].shift(45)
test['lag_5'] = test['visitors'].shift(44)
test['lag_6'] = test['visitors'].shift(43)

In [40]:
#Moving averages
train['last2_day_MA'] = train[['lag_1', 'lag_2']].mean(axis=1)
train['last3_day_MA'] = train[['lag_1', 'lag_2','lag_3']].mean(axis=1)
train['last5_day_MA'] = train[['lag_1', 'lag_2','lag_3','lag_4','lag_5']].mean(axis=1)
test['last2_day_MA'] = test[['lag_1', 'lag_2']].mean(axis=1)
test['last3_day_MA'] = test[['lag_1', 'lag_2','lag_3']].mean(axis=1)
test['last5_day_MA'] = test[['lag_1', 'lag_2','lag_3','lag_4','lag_5']].mean(axis=1)

In [41]:
#last week average
train['lag_a'] = train['visitors'].shift(1)
train['lag_b'] = train['visitors'].shift(2)
train['lag_c'] = train['visitors'].shift(3)
train['lag_d'] = train['visitors'].shift(4)
train['lag_e'] = train['visitors'].shift(5)
train['lag_f'] = train['visitors'].shift(6)
train['lag_g'] = train['visitors'].shift(7)
train['last_week_avg'] = train[['lag_a', 'lag_b','lag_c','lag_d','lag_e','lag_f','lag_g']].mean(axis=1)
train = train.drop(['lag_a', 'lag_b','lag_c','lag_d','lag_e','lag_f','lag_g'],axis=1)

In [42]:
#last week's average - Last feature
temp1 = train.loc[(train['date_int'] >= 20170416)&(train['date_int'] <=20170422 )]
temp = temp1.groupby(['air_store_id'],as_index=False)['visitors'].mean().rename(columns={'visitors':'last_week_avg'})
test = pd.merge(test,temp,on = ('air_store_id'),how='left')

In [43]:
train.head(2)

,air_store_id,visit_date,visitors,dow,doy,year,month,week,day_of_week,holiday_flg,min_visitors,mean_visitors,median_visitors,count_observations,air_genre_name,air_area_name,latitude,longitude,air_genre_name0,air_area_name0,air_genre_name1,air_area_name1,air_genre_name2,air_area_name2,air_genre_name3,air_area_name3,air_genre_name4,air_area_name4,air_genre_name5,air_area_name5,air_genre_name6,air_area_name6,air_genre_name7,air_area_name7,air_genre_name8,air_area_name8,air_genre_name9,air_area_name9,rs1_x,rv1_x,rs2_x,rv2_x,rs1_y,rv1_y,rs2_y,rv2_y,id,total_reserv_sum,total_reserv_mean,total_reserv_dt_diff_mean,date_int,var_max_lat,var_max_long,lon_plus_lat,air_store_id2,date,dly_avg,dly_median,dly_min,dly_max,lat-lon,lat/lon,latintolon,mnth_avg,mnth_median,mnth_min,mnth_max,mnthly-avg_dly,restaurant_avg,restaurant_median,restaurant_min,restaurant_max,avg_dly_mnth,mnth_dt_avg_by_area,mnth_dt_median_by_area,mnth_dt_min_by_area,mnth_dt_max_by_area,dow_avg,avg_vstr_wkday_by_area,median_vstr_wkday_by_area,min_vstr_wkday_by_area,max_vstr_wkday_by_area,avg_vstr_wkday_by_cuisine,median_vstr_wkday_by_cuisine,min_vstr_wkday_by_cuisine,max_vstr_wkday_by_cuisine,avgofavg,medianofavg,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,restaurants_by_area,rating,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,last2_day_MA,last3_day_MA,last5_day_MA,last_week_avg
92314,air_00a91d42b08b08d9,2016-07-01,35,4,183,2016,7,26,0,0,17.0,36.500000,35.5,40.0,6,44,35.694003,139.753595,6,7,2,6,0,3,0,6,0,48,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,air_00a91d42b08b08d9_2016-07-01,NaN,NaN,NaN,20160701,8.326629,4.519803,175.447598,0,1,27.0,27.0,7,56,-104.059592,0.255407,4988.365201,26.08,25.0,5,53,-0.918103,26.081897,26.0,1,99,0.001897,33.277778,35.0,5,68,36.500000,33.967033,31.0,1,93,25.658881,22.0,1,244,28.915474,26.540948,NaN,NaN,NaN,NaN,NaN,NaN,35.0,16.0,24.0,56.0,22.0,39.0,NaN,27.0,17.0,7.0,20,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
92288,air_00a91d42b08b08d9,2016-07-02,9,5,184,2016,7,26,2,0,3.0,14.973684,11.0,38.0,6,44,35.694003,139.753595,6,7,2,6,0,3,0,6,0,48,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,air_00a91d42b08b08d9_2016-07-02,NaN,NaN,NaN,20160702,8.326629,4.519803,175.447598,0,2,25.0,25.0,9,35,-104.059592,0.255407,4988.365201,26.08,25.0,5,53,1.081897,26.081897,26.0,1,99,0.001897,16.461538,9.0,2,51,14.973684,17.459459,11.0,1,76,26.584733,23.0,1,167,23.033895,25.540000,NaN,NaN,NaN,NaN,NaN,NaN,9.0,24.0,26.0,NaN,24.0,35.0,NaN,24.0,33.0,NaN,20,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35.0


In [44]:
train.head(2)

,air_store_id,visit_date,visitors,dow,doy,year,month,week,day_of_week,holiday_flg,min_visitors,mean_visitors,median_visitors,count_observations,air_genre_name,air_area_name,latitude,longitude,air_genre_name0,air_area_name0,air_genre_name1,air_area_name1,air_genre_name2,air_area_name2,air_genre_name3,air_area_name3,air_genre_name4,air_area_name4,air_genre_name5,air_area_name5,air_genre_name6,air_area_name6,air_genre_name7,air_area_name7,air_genre_name8,air_area_name8,air_genre_name9,air_area_name9,rs1_x,rv1_x,rs2_x,rv2_x,rs1_y,rv1_y,rs2_y,rv2_y,id,total_reserv_sum,total_reserv_mean,total_reserv_dt_diff_mean,date_int,var_max_lat,var_max_long,lon_plus_lat,air_store_id2,date,dly_avg,dly_median,dly_min,dly_max,lat-lon,lat/lon,latintolon,mnth_avg,mnth_median,mnth_min,mnth_max,mnthly-avg_dly,restaurant_avg,restaurant_median,restaurant_min,restaurant_max,avg_dly_mnth,mnth_dt_avg_by_area,mnth_dt_median_by_area,mnth_dt_min_by_area,mnth_dt_max_by_area,dow_avg,avg_vstr_wkday_by_area,median_vstr_wkday_by_area,min_vstr_wkday_by_area,max_vstr_wkday_by_area,avg_vstr_wkday_by_cuisine,median_vstr_wkday_by_cuisine,min_vstr_wkday_by_cuisine,max_vstr_wkday_by_cuisine,avgofavg,medianofavg,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,restaurants_by_area,rating,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,last2_day_MA,last3_day_MA,last5_day_MA,last_week_avg
92314,air_00a91d42b08b08d9,2016-07-01,35,4,183,2016,7,26,0,0,17.0,36.500000,35.5,40.0,6,44,35.694003,139.753595,6,7,2,6,0,3,0,6,0,48,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,air_00a91d42b08b08d9_2016-07-01,NaN,NaN,NaN,20160701,8.326629,4.519803,175.447598,0,1,27.0,27.0,7,56,-104.059592,0.255407,4988.365201,26.08,25.0,5,53,-0.918103,26.081897,26.0,1,99,0.001897,33.277778,35.0,5,68,36.500000,33.967033,31.0,1,93,25.658881,22.0,1,244,28.915474,26.540948,NaN,NaN,NaN,NaN,NaN,NaN,35.0,16.0,24.0,56.0,22.0,39.0,NaN,27.0,17.0,7.0,20,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
92288,air_00a91d42b08b08d9,2016-07-02,9,5,184,2016,7,26,2,0,3.0,14.973684,11.0,38.0,6,44,35.694003,139.753595,6,7,2,6,0,3,0,6,0,48,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,air_00a91d42b08b08d9_2016-07-02,NaN,NaN,NaN,20160702,8.326629,4.519803,175.447598,0,2,25.0,25.0,9,35,-104.059592,0.255407,4988.365201,26.08,25.0,5,53,1.081897,26.081897,26.0,1,99,0.001897,16.461538,9.0,2,51,14.973684,17.459459,11.0,1,76,26.584733,23.0,1,167,23.033895,25.540000,NaN,NaN,NaN,NaN,NaN,NaN,9.0,24.0,26.0,NaN,24.0,35.0,NaN,24.0,33.0,NaN,20,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35.0


In [45]:
#Important Features from Boruta
train = train[['visitors',u'visit_date',u'dow',u'doy', u'month', u'week', u'day_of_week',
       u'holiday_flg', u'min_visitors', u'median_visitors', u'air_genre_name1',
       u'air_genre_name3', u'air_genre_name9', u'rs1_x', u'id', u'date_int',
       u'var_max_lat', u'var_max_long', u'lon_plus_lat', u'air_store_id2',
       u'date', u'dly_avg', u'dly_median', u'dly_max', u'lat-lon', u'lat/lon',
       u'latintolon', u'mnth_avg', u'mnth_median', u'mnth_min',
       u'mnthly-avg_dly', u'restaurant_avg', u'restaurant_median',
       u'restaurant_min', u'restaurant_max', u'avg_dly_mnth',
       u'mnth_dt_avg_by_area', u'mnth_dt_median_by_area',
       u'mnth_dt_min_by_area', u'dow_avg', u'avg_vstr_wkday_by_area',
       u'median_vstr_wkday_by_area', u'max_vstr_wkday_by_area',
       u'avg_vstr_wkday_by_cuisine', u'median_vstr_wkday_by_cuisine',
       u'min_vstr_wkday_by_cuisine', u'max_vstr_wkday_by_cuisine', u'avgofavg',
       u'medianofavg', u'a', u'b', u'c', u'd', u'e',
       u'f', u'g', u'h', u'i', u'j', u'k',
       u'l', u'm', u'o', u'p', u'restaurants_by_area',
       u'rating', u'lag_1', u'lag_2', u'lag_3', u'lag_4', u'lag_5', u'lag_6']]

In [46]:
train.to_csv("/Users/mayanksoni/Desktop/Kaggle /Restaurant Visitor Forecasting/Final/train.csv")
test.to_csv("/Users/mayanksoni/Desktop/Kaggle /Restaurant Visitor Forecasting/Final/test.csv")

In [47]:
y_train = np.log1p(train['visitors'].values)

In [48]:
train = train.fillna(-1)
test = test.fillna(-1)

In [49]:
#split into valid and train
from sklearn.cross_validation import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(train, y_train, test_size=0.15, random_state=4242)

In [50]:
col = [c for c in train if c not in ['id', 'air_store_id', 'visit_date','visitors']]

In [51]:
print('Building DMatrix...')

d_train = xgb.DMatrix(x_train[col], label=y_train)
d_valid = xgb.DMatrix(x_valid[col], label=y_valid)

Building DMatrix...


In [54]:
print('Training ...')

params = {}
params['objective'] = 'reg:linear'
params['eval_metric'] = 'rmse'
params['learning_rate']=0.04
params['gamma'] = 1
params['min_child_weight'] = 2
params['subsample'] = 1
params['colsample_bytree'] = 0.80
params['scale_pos_weight'] = 1
params['seed'] = 27
#params['eta'] = 0.04
params['max_depth'] = 6
params['silent'] = 1
watchlist = [(d_train, 'train'), (d_valid, 'valid')]
clf = xgb.train(params, d_train, 1000, watchlist, early_stopping_rounds=100, verbose_eval=10)

Training ...
[0]	train-rmse:2.347	valid-rmse:2.34047
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 100 rounds.
[10]	train-rmse:1.60111	valid-rmse:1.59663
[20]	train-rmse:1.11992	valid-rmse:1.11751
[30]	train-rmse:0.818768	valid-rmse:0.818661
[40]	train-rmse:0.638985	valid-rmse:0.641417
[50]	train-rmse:0.537642	valid-rmse:0.542529
[60]	train-rmse:0.483734	valid-rmse:0.49036
[70]	train-rmse:0.45547	valid-rmse:0.463599
[80]	train-rmse:0.439475	valid-rmse:0.448667
[90]	train-rmse:0.427852	valid-rmse:0.437977
[100]	train-rmse:0.41931	valid-rmse:0.430151
[110]	train-rmse:0.413255	valid-rmse:0.424681
[120]	train-rmse:0.409123	valid-rmse:0.420914
[130]	train-rmse:0.405	valid-rmse:0.417215
[140]	train-rmse:0.401835	valid-rmse:0.414533
[150]	train-rmse:0.39879	valid-rmse:0.411993
[160]	train-rmse:0.395286	valid-rmse:0.408943
[170]	train-rmse:0.391966	valid-rmse:0.406041
[180]	train-rmse:0.38908	valid-rmse:0.4

In [57]:
d_test = xgb.DMatrix(test[col])

In [58]:
print('Predicting on test ...')

p_test = clf.predict(d_test)

Predicting on test ...


In [59]:
test['visitors'] = np.expm1(p_test)

test[['id','visitors']].to_csv('last.csv', index=False, float_format='%.3f')